In [0]:
%sql
USE CATALOG catproject_catalog;

In [0]:
%sql
USE SCHEMA ecom_bronze;

In [0]:
%sql
CREATE TABLE ecom_bronze.payment_raw (
  payment_id           STRING,         -- Mã dòng tiền
  payment_type         STRING,         -- inbound / outbound
  payment_category     STRING,         -- sale, purchase, salary, tax, refund, etc.
  invoice_number       STRING,         -- Mã hóa đơn liên quan
  invoice_type         STRING,         -- import / export
  partner_id           STRING,         -- Mã khách hàng hoặc nhà cung cấp
  partner_name         STRING,
  amount               DECIMAL(18, 2), -- Số tiền giao dịch
  currency             STRING,
  payment_method       STRING,         -- cash, bank_transfer, credit_card, etc.
  payment_status       STRING,         -- pending, completed, failed
  payment_date         DATE,
  description          STRING,
  source_system        STRING,
  ingestion_time       TIMESTAMP,
  ingestion_year       INT,
  ingestion_month      INT,
  operation_type       STRING,
  record_hash          STRING,
  source_updated_at    TIMESTAMP
)
USING DELTA
PARTITIONED BY (ingestion_year, ingestion_month)
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true')

Tạo Comment cho các cột

In [0]:
%sql
COMMENT ON COLUMN ecom_bronze.payment_raw.payment_id IS 'Mã định danh duy nhất cho dòng tiền';
COMMENT ON COLUMN ecom_bronze.payment_raw.payment_type IS 'Phân loại dòng tiền: inbound (thu) hoặc outbound (chi)';
COMMENT ON COLUMN ecom_bronze.payment_raw.payment_category IS 'Danh mục của dòng tiền: sale, purchase, salary, tax, refund,...';
COMMENT ON COLUMN ecom_bronze.payment_raw.invoice_number IS 'Mã số hóa đơn liên quan đến giao dịch';
COMMENT ON COLUMN ecom_bronze.payment_raw.invoice_type IS 'Loại hóa đơn: import (hóa đơn mua) / export (hóa đơn bán)';
COMMENT ON COLUMN ecom_bronze.payment_raw.partner_id IS 'Đối tác liên quan đến thanh toán (khách hàng/nhà cung cấp)';
COMMENT ON COLUMN ecom_bronze.payment_raw.partner_name IS 'Tên đối tác thanh toán';
COMMENT ON COLUMN ecom_bronze.payment_raw.amount IS 'Số tiền giao dịch';
COMMENT ON COLUMN ecom_bronze.payment_raw.currency IS 'Loại tiền tệ sử dụng';
COMMENT ON COLUMN ecom_bronze.payment_raw.payment_method IS 'Phương thức thanh toán';
COMMENT ON COLUMN ecom_bronze.payment_raw.payment_status IS 'Trạng thái giao dịch';
COMMENT ON COLUMN ecom_bronze.payment_raw.payment_date IS 'Ngày thực hiện thanh toán';
COMMENT ON COLUMN ecom_bronze.payment_raw.description IS 'Mô tả chi tiết dòng tiền';
COMMENT ON COLUMN ecom_bronze.payment_raw.source_system IS 'Hệ thống nguồn ghi nhận dữ liệu';
COMMENT ON COLUMN ecom_bronze.payment_raw.ingestion_time IS 'Thời điểm đẩy dữ liệu lên warehouse';
COMMENT ON COLUMN ecom_bronze.payment_raw.operation_type IS 'Loại thao tác ghi nhận: insert, update, delete';
COMMENT ON COLUMN ecom_bronze.payment_raw.record_hash IS 'Hash dùng để kiểm tra thay đổi bản ghi';
COMMENT ON COLUMN ecom_bronze.payment_raw.source_updated_at IS 'Thời điểm cập nhật cuối cùng từ hệ thống nguồn';


In [0]:
%sql
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN payment_id SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN payment_type SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN payment_category SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN invoice_number SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN invoice_type SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN partner_id SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN partner_name SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN amount SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN currency SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN payment_method SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN payment_status SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN payment_date SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN description SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN source_system SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN ingestion_time SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN ingestion_year SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN ingestion_month SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN operation_type SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN record_hash SET NOT NULL;
ALTER TABLE ecom_bronze.payment_raw ALTER COLUMN source_updated_at SET NOT NULL;

-- payment_type phải là inbound hoặc outbound
ALTER TABLE ecom_bronze.payment_raw
ADD CONSTRAINT valid_payment_type
CHECK (payment_type IN ('inbound', 'outbound'));

-- payment_status chỉ chấp nhận một số giá trị xác định
ALTER TABLE ecom_bronze.payment_raw
ADD CONSTRAINT valid_payment_status
CHECK (payment_status IN ('pending', 'completed', 'failed'));

-- currency phải hợp lệ (ví dụ)
ALTER TABLE ecom_bronze.payment_raw
ADD CONSTRAINT valid_currency
CHECK (currency IN ('USD', 'VND'));

-- payment_date không thể ở tương lai
ALTER TABLE ecom_bronze.payment_raw
ADD CONSTRAINT valid_payment_date
CHECK (payment_date <= current_date());

-- amount phải >= 0
ALTER TABLE ecom_bronze.payment_raw
ADD CONSTRAINT valid_amount
CHECK (amount >= 0);

-- Kiểm tra cách thanh toán hợp lệ
ALTER TABLE ecom_bronze.payment_raw
ADD CONSTRAINT valid_payment_method
CHECK (
  payment_method IN ('cash', 'credit_card', 'bank_transfer', 'e_wallet')
);

-- ingestion_time phải nhỏ hơn thời gian hiện tại
ALTER TABLE ecom_bronze.payment_raw
ADD CONSTRAINT valid_ingestion_time
CHECK (ingestion_time < current_timestamp());

